# Deep-Learning with Keras

#### Ugur URESIN, AI Engineer | Data Scientist
#### Mail: uresin.ugur@gmail.com

## Chapter 05. Neural Networks - Regression
### Dataset: Boston Housing

### The Steps
* **Step01**. Loading Dataset
* **Step02**. Assesing the Dataset
* **Step03**. Preparing the Dataset
* **Step04**. Building the network architecture
* **Step05**. The compilation
* **Step06**. Creating a Validation Set
* **Step07**. Training
* **Step08**. Evaluating the performance

### Step 01. Loading Dataset

In [1]:
from keras.datasets import boston_housing

In [9]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

### Step 02. Assesing the Dataset

In [10]:
print("Training data dimensions:", train_data.shape)
print("Number of training labels:", len(train_targets))
print("Test data dimensions:", test_data.shape)
print("Number of test labels:", len(test_targets))

Training data dimensions: (404, 13)
Number of training labels: 404
Test data dimensions: (102, 13)
Number of test labels: 102


### Step03. Preparing the Dataset

#### Normalizing the datasets 

It would be problematic to feed into a neural network values that all take wildly different ranges.  
The network might be able to automatically adapt to such heterogeneous data, but it would definitely make learning more difficult.  
A widespread best practice to deal with such data is to do **feature-wise normalization**.  

For each feature in the input data (a column in the input data matrix),  
the mean of the feature is substracted and divided by the standard deviation,  
so that the feature is centered around 0 and has a unit standard deviation.

In [11]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

Note that the quantities used **for normalizing the test data** are computed **using the
training data**!!!  

You should **never use any quantity computed on the test data**, even for something as simple as data normalization.

### Step04. Building the network architecture

In [12]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae']) #mae:mean-absolute-error
    return model

The network ends with a single unit and **no activation** (it will be a linear layer).   
This is a typical setup for scalar regression (a regression where you’re trying to predict a single continuous value).

A new metric is monitored during training: **mean absolute error (MAE)**.  
It’s the absolute value of the difference between the predictions and the targets.  
For instance, an MAE of 0.5 on this problem would mean your predictions are off by $500 on average.

### Step06. The Compilation and Training

#### k-fold validation

![k-fold validation](./img/kfold_Validation.png "k-fold validation")

In [18]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i+1)
    
    #preparing the validation data, from k'th partition of the data
    val_data    = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    
    #preparing the training data: data from all other partitions
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],
                                         train_data[(i + 1) * num_val_samples:]],
                                        axis=0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                            train_targets[(i + 1) * num_val_samples:]],
                                           axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4


In [19]:
print("Scores: ", all_scores)
print("Mean score: ", np.mean(all_scores))

Scores:  [2.086811065673828, 2.220487117767334, 2.6658310890197754, 2.27522349357605]
Mean score:  2.312088191509247
